In [20]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.0.1+cu118
0.15.2+cu118


In [21]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torchinfo import summary

In [22]:
try:
    from going_modular import data_setup, engine
except:
    print("downloading...")
    print()
    !git clone https: // github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular.
    !rm -rf pytorch-deep-learning
    from going_modular import data_setup, engine

    print("done")

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [24]:
def set_seeds(seed: int = 42):
    """sets random sets for torch operation
    
    Args:
         seed (int, optional) : Random seed to set. default 42."""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [25]:
import os
import zipfile
from pathlib import Path
import requests


# our source: https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    """download a zipped file from the source and unzips it to the destination."""
    # setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # if the image folder doesn't exist, create it
    if image_path.is_dir():
        print(f"[info] {image_path} directory already exit, skipping download.")
    else:
        print(f"[info] did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # download the target data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[info] downloading {target_file} from {source} ...")
            f.write(request.content)

        # unzipping
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[info] Unzipping {target_file} data ...")
            zip_ref.extractall(image_path)

        # removing source
        if remove_source:
            os.remove(data_path / target_file)
    return image_path


image_path = download_data(
    source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
    destination="pizza_steak_sushi",
    remove_source=True)
image_path

[info] data\pizza_steak_sushi directory already exit, skipping download.


WindowsPath('data/pizza_steak_sushi')

### create DataLoader with manual transform

In [26]:
## create DataLoader with manual transform
# setup directories
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [27]:
# setup imagenet normalization

from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# create transforms pipeline manually
manual_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                        transforms.ToTensor(),
                                        normalize])
print(manual_transforms)
from going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms,
                                                                               test_transform=manual_transforms,
                                                                               batch_size=32,
                                                                               num_workers=0)
train_dataloader, test_dataloader, class_names

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x26d8b152510>,
 ['pizza', 'steak', 'sushi'])

### automaticlly method 

In [28]:
train_dir = image_path/"train"
test_dir = image_path/"test"
import torchvision
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # means best available
automatic_transforms = weights.transforms()
print(automatic_transforms)

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=automatic_transforms,
                                                                               test_transform=automatic_transforms,
                                                                               batch_size=32,
                                                                               num_workers=0)
train_dataloader, test_dataloader, class_names

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x26d8b13d810>,
 ['pizza', 'steak', 'sushi'])

### getting pretrained model

In [29]:
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) OLD
torch.hub.set_dir("data")
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [30]:
# freeze all base layers by setting required_grad to false.
for param in model.parameters():
    param.requires_grad = False

In [31]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [32]:
# adjust classifier
set_seeds()
model.classifier = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                                 nn.Linear(in_features=1280, out_features=3, bias=True)).to(device)

In [33]:
from torchinfo import summary
summary(model,
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"],
        verbose=0)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [34]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


In [35]:
# set up a summery writer
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
writer

ModuleNotFoundError: No module named 'tensorboard'

In [36]:
from torch.utils.tensorboard import SummaryWriter

ModuleNotFoundError: No module named 'tensorboard'